In [1]:
import pandas as pd
import folium
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from folium.plugins import HeatMap
from global_land_mask import globe
from collections import defaultdict
import geopandas as gpd
import pygeohash as pgh
import seaborn as sns
from shapely.geometry import Polygon
import branca.colormap
from matplotlib import rcParams
from functools import reduce

In [2]:
 x = pd.read_csv("2018_OilTankers.csv")[["imo","hour","long_x","lat_x","nav_status_x"]]
 x

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,imo,hour,long_x,lat_x,nav_status_x
0,4902957,0,-99999.000000,-99999.000000,-99999
1,4902957,1,-99999.000000,-99999.000000,-99999
2,4902957,2,-99999.000000,-99999.000000,-99999
3,4902957,3,-99999.000000,-99999.000000,-99999
4,4902957,4,-99999.000000,-99999.000000,-99999
...,...,...,...,...,...
46070451,9876543,8755,9.675287,23.076577,Under Way Using Engine
46070452,9876543,8756,-64.597708,10.251247,Under Way Using Engine
46070453,9876543,8757,9.675287,10.251245,Under Way Using Engine
46070454,9876543,8758,9.675288,31.764420,Not Defined


In [4]:
x = x[x["long_x"]!= -99999.0][x["lat_x"]!= -99999.0][x["nav_status_x"]!= "-99999"]
x

<ipython-input-4-f3e4cda48cda>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = x[x["long_x"]!= -99999.0][x["lat_x"]!= -99999.0][x["nav_status_x"]!= "-99999"]


,imo,hour,long_x,lat_x,nav_status_x
3513,4902957,3513,9.189310,3.999807,Underway Sailing
4306,4902957,4306,9.192333,4.002607,Underway Sailing
4307,4902957,4307,9.191465,3.999332,Underway Sailing
4310,4902957,4310,9.191472,3.999381,Underway Sailing
4311,4902957,4311,9.190638,4.000052,Underway Sailing
...,...,...,...,...,...
46070451,9876543,8755,9.675287,23.076577,Under Way Using Engine
46070452,9876543,8756,-64.597708,10.251247,Under Way Using Engine
46070453,9876543,8757,9.675287,10.251245,Under Way Using Engine
46070454,9876543,8758,9.675288,31.764420,Not Defined


In [5]:
x["nav_status_x"].unique()

array(['Underway Sailing', 'Under Way Using Engine', 'Moored',
       'Not Defined', 'Unknown', 'At Anchor', 'Not Under Command',
       'Engaged In Fishing', 'Aground', 'Restricted Manoeuvrability'],
      dtype=object)

In [6]:
x["nav_status_x"].value_counts()

Under Way Using Engine        14979456
At Anchor                      6715329
Moored                         3239797
Underway Sailing                545489
Not Defined                     300088
Not Under Command               230937
Unknown                         201926
Restricted Manoeuvrability      173105
Aground                           3835
Engaged In Fishing                1874
Name: nav_status_x, dtype: int64

In [7]:
x = x[["imo","lat_x","long_x","nav_status_x"]].dropna().sample(100000)
x

,imo,lat_x,long_x,nav_status_x
4948270,8904135,6.951698,79.839768,Under Way Using Engine
37288755,9635078,1.299772,103.971882,At Anchor
15974854,9241114,7.671667,109.331667,Under Way Using Engine
39201281,9691242,35.513194,140.041406,Moored
35540548,9579523,18.691198,-20.118611,Under Way Using Engine
...,...,...,...,...
16819454,9251810,22.850000,-93.742000,Under Way Using Engine
9901480,9123075,5.231667,98.308333,Under Way Using Engine
29412215,9407275,40.836987,29.280783,Moored
6251642,9031507,-2.244033,-104.075417,Under Way Using Engine


In [8]:
x["nav_status_x"].value_counts()

Under Way Using Engine        56955
At Anchor                     25342
Moored                        12182
Underway Sailing               2098
Not Defined                    1115
Not Under Command               866
Unknown                         782
Restricted Manoeuvrability      643
Aground                          12
Engaged In Fishing                5
Name: nav_status_x, dtype: int64

In [9]:


#create a map
this_map = folium.Map(prefer_canvas=True)


def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.lat_x, point.long_x],
                        radius=2,
                        weight=0,#remove outline
                        popup = point.nav_status_x,
                        fill_color='#F08080').add_to(this_map)

#use df.apply(,axis=1) to iterate through every row in your dataframe
x.apply(plotDot, axis = 1)

#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file


this_map
import os

this_map.save(os.path.join("/Users/zwt2000/Desktop/Ship_Visualisation/","Locationbynavstatus.html"))
